In [ ]:
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as si 
import tweepy
import time

In [ ]:
#gets env variables
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
BRR_TOKEN = os.getenv('BRR_TOKEN')

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


In [ ]:
while True:
    Archive = pd.read_csv('Archive.csv',index_col=[0])
    open_pos = Archive[Archive['Price Out'].isnull()]
    list_of_open = list(open_pos.index.values) #list of index locations of open positions
    print(list_of_open)
    pos_2_close = []
    for index in list_of_open:
        Symbol = Archive.iloc[index,0]
        price_sym = si.get_live_price(Symbol)
        curr_price_diff = price_sym - Archive.iloc[index,2]
        if curr_price_diff/Archive.iloc[index,2] >= 0.05:
            pos_2_close.append(index)
        elif curr_price_diff/Archive.iloc[index,2] <= -0.03:
            pos_2_close.append(index)
        else:
            continue
    
    if pos_2_close != []:
        for index in pos_2_close:
            Symbol = Archive.iloc[index,0]
            price_out = si.get_live_price(Symbol)
            bot_says = f'Closing position on {Symbol} at ${price_out} This is not financial advice.'
            response = api.update_status(status=bot_says)
            print(response)
            Archive.at[index,'Price Out'] = price_out #Price out value 
    else:
        pass
    Archive['%Change'] = (Archive['Price Out'] - Archive['Price'])/Archive['Price']*100
    Archive['W/L'] = np.select([Archive['Price Out']>Archive['Price'],Archive['Price Out']<Archive['Price']],
                        choicelist=[1, 0],
                        default= np.nan)
    print(pos_2_close)
    Archive.to_csv('Archive.csv')
    time.sleep(30)  
